# 作业

更换数据集MSRA和ERNIE-Gram或BERT等预训练模型。

- 数据集：
`train_ds, test_ds = load_dataset("msra_ner", splits=["train", "test"])`
- 模型：
	将`from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification`换成相应的模型。

# 使用PaddleNLP语义预训练模型ERNIE完成快递单信息抽取


**注意**

本项目代码需要使用GPU环境来运行:

<img src="https://ai-studio-static-online.cdn.bcebos.com/767f625548714f03b105b6ccb3aa78df9080e38d329e445380f505ddec6c7042" width="40%" height="40%">
<br>
<br>

命名实体识别是NLP中一项非常基础的任务，是信息提取、问答系统、句法分析、机器翻译等众多NLP任务的重要基础工具。命名实体识别的准确度，决定了下游任务的效果，是NLP中的一个基础问题。在NER任务提供了两种解决方案，一类LSTM/GRU + CRF，通过RNN类的模型来抽取底层文本的信息，而CRF(条件随机场)模型来学习底层Token之间的联系；另外一类是通过预训练模型，例如ERNIE，BERT模型，直接来预测Token的标签信息。

本项目将演示如何使用PaddleNLP语义预训练模型ERNIE完成从快递单中抽取姓名、电话、省、市、区、详细地址等内容，形成结构化信息。辅助物流行业从业者进行有效信息的提取，从而降低客户填单的成本。

在2017年之前，工业界和学术界对文本处理依赖于序列模型[Recurrent Neural Network (RNN)](https://baike.baidu.com/item/%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/23199490?fromtitle=RNN&fromid=5707183&fr=aladdin).

<p align="center">
<img src="http://colah.github.io/posts/2015-09-NN-Types-FP/img/RNN-general.png" width="40%" height="30%"> <br />
</p><br><center>图1：RNN示意图</center></br>

[基于BiGRU+CRF的快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)项目介绍了如何使用序列模型完成快递单信息抽取任务。
<br>

近年来随着深度学习的发展，模型参数的数量飞速增长。为了训练这些参数，需要更大的数据集来避免过拟合。然而，对于大部分NLP任务来说，构建大规模的标注数据集非常困难（成本过高），特别是对于句法和语义相关的任务。相比之下，大规模的未标注语料库的构建则相对容易。为了利用这些数据，我们可以先从其中学习到一个好的表示，再将这些表示应用到其他任务中。最近的研究表明，基于大规模未标注语料库的预训练模型（Pretrained Models, PTM) 在NLP任务上取得了很好的表现。

近年来，大量的研究表明基于大型语料库的预训练模型（Pretrained Models, PTM）可以学习通用的语言表示，有利于下游NLP任务，同时能够避免从零开始训练模型。随着计算能力的不断提高，深度模型的出现（即 Transformer）和训练技巧的增强使得 PTM 不断发展，由浅变深。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/327f44ff3ed24493adca5ddc4dc24bf61eebe67c84a6492f872406f464fde91e" width="60%" height="50%"> <br />
</p><br><center>图2：预训练模型一览，图片来源于：https://github.com/thunlp/PLMpapers</center></br>
                                                                                                                             
本示例展示了以ERNIE([Enhanced Representation through Knowledge Integration](https://arxiv.org/pdf/1904.09223))为代表的预训练模型如何Finetune完成序列标注任务。

**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

开源不易，希望大家多多支持~ 

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装。

In [ ]:
!pip install --upgrade paddlenlp

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 12.0MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [1]:
from functools import partial

import paddle
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.transformers import BertTokenizer, BertForTokenClassification
from paddlenlp.metrics import ChunkEvaluator
from utils import convert_my_example, evaluate, my_predict, load_dict

## 加载MSRA数据集

推荐使用load_dataset()加载MSRA数据集。

In [2]:
from paddlenlp.datasets import load_dataset

# 由于MSRA_NER数据集没有dev dataset，这里使用test dataset作为dev dataset
train_ds, dev_ds, test_ds = load_dataset(
        'msra_ner', splits=('train', 'test', 'test'), lazy=False)


In [ ]:
for i in range(5):
    print(train_ds[i])

{'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'labels': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]}
{'tokens': ['藏', '书', '本', '来', '就', '是', '所', '有', '传', '统', '收', '藏', '门', '类', '中', '的', '第', '一', '大', '户', '，', '只', '是', '我', '们', '结', '束', '温', '饱', '的', '时', '间', '太', '短', '而', '已', '。'], 'labels': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]}
{'tokens': ['因', '有', '关', '日', '寇', '在', '京', '掠', '夺', '文', '物', '详', '情', '，', '藏', '界', '较', '为', '重', '视', '，', '也', '是', '我', '们', '收', '藏', '北', '京', '史', '料', '中', '的', '要', '件', '之', '一', '。'], 'labels': [6, 6, 6, 4, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6

每条数据包含一句文本和这个文本中每个汉字以及数字对应的label标签。

之后，还需要对输入句子进行数据处理，如切词，映射词表id等。

## 数据处理

预训练模型ERNIE对中文数据的处理是以字为单位。PaddleNLP对于各种预训练模型已经内置了相应的tokenizer。指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer作用为将原始输入文本转化成模型model可以接受的输入数据形式。


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>
<br><center>图3：ERNIE模型示意图</center></br>

In [3]:
label_vocab = {label:label_id for label_id, label in enumerate(train_ds.label_list)}
#tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 这里的convert_my_example是依托MSRA数据集特点修改后的
trans_func = partial(convert_my_example, tokenizer=tokenizer, label_vocab=label_vocab)

train_ds.map(trans_func)
dev_ds.map(trans_func)
test_ds.map(trans_func)
print (train_ds[0])

[2021-06-12 11:46:08,877] [    INFO] - Found /home/aistudio/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt


([101, 2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 52, [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])


### 数据读入

使用`paddle.io.DataLoader`接口多线程异步加载数据。

In [4]:
ignore_label = -1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=ignore_label)  # labels
): fn(samples)

# 为了不浪费至尊卡，增大了batch_size
train_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_size=128,
    return_list=True,
    collate_fn=batchify_fn)
dev_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_size=128,
    return_list=True,
    collate_fn=batchify_fn)
test_loader = paddle.io.DataLoader(
    dataset=test_ds,
    batch_size=128,
    return_list=True,
    collate_fn=batchify_fn)

## PaddleNLP一键加载预训练模型


快递单信息抽取本质是一个序列标注任务，PaddleNLP对于各种预训练模型已经内置了对于下游任务文本分类Fine-tune网络。以下教程以ERNIE为预训练模型完成序列标注任务。

`paddlenlp.transformers.ErnieForTokenClassification()`一行代码即可加载预训练模型ERNIE用于序列标注任务的fine-tune网络。其在ERNIE模型后拼接上一个全连接网络进行分类。

`paddlenlp.transformers.ErnieForTokenClassification.from_pretrained()`方法只需指定想要使用的模型名称和文本分类的类别数即可完成定义模型网络。

In [5]:
# Define the model netword and its loss
#model = ErnieForTokenClassification.from_pretrained("ernie-1.0", num_classes=len(label_vocab))
model = BertForTokenClassification.from_pretrained("bert-base-chinese", num_classes=len(label_vocab))
#state_dict = paddle.load("bert_result/model_3520.pdparams")
#model.set_dict(state_dict)

PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa、Electra等预训练模型。
下表汇总了目前PaddleNLP支持的各类预训练模型。您可以使用PaddleNLP提供的模型，完成文本分类、序列标注、问答等任务。同时我们提供了众多预训练模型的参数权重供用户使用，其中包含了二十多种中文语言模型的预训练权重。中文的预训练模型有`bert-base-chinese, bert-wwm-chinese, bert-wwm-ext-chinese, ernie-1.0, ernie-tiny, gpt2-base-cn, roberta-wwm-ext, roberta-wwm-ext-large, rbt3, rbtl3, chinese-electra-base, chinese-electra-small, chinese-xlnet-base, chinese-xlnet-mid, chinese-xlnet-large, unified_transformer-12L-cn, unified_transformer-12L-cn-luge`等。

更多预训练模型参考：[PaddleNLP Transformer API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md)。

更多预训练模型fine-tune下游任务使用方法，请参考：[examples](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples)。

## 设置Fine-Tune优化策略，模型配置
适用于ERNIE/BERT这类Transformer模型的迁移优化学习率策略为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p><br><center>图4：动态学习率示意图</center></br>



In [ ]:
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
loss_fn = paddle.nn.loss.CrossEntropyLoss(ignore_index=ignore_label)
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())

## 模型训练与评估


模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。将前向计算结果传给评价方法，计算评价指标。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch时，程序将会评估一次，评估当前模型训练的效果。

In [ ]:
step = 0
for epoch in range(10):
    for idx, (input_ids, token_type_ids, length, labels) in enumerate(train_loader):
        logits = model(input_ids, token_type_ids)
        loss = paddle.mean(loss_fn(logits, labels))
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        step += 1
        if step%100==0:
            print("epoch:%d - step:%d - loss: %f" % (epoch, step, loss))
    evaluate(model, metric, dev_loader)

    paddle.save(model.state_dict(),
                './bert_result/model_%d.pdparams' % step)
# model.save_pretrained('./checkpoint')
# tokenizer.save_pretrained('./checkpoint')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


epoch:0 - step:100 - loss: 0.094581
epoch:0 - step:200 - loss: 0.042849
epoch:0 - step:300 - loss: 0.025877


[2021-06-12 10:55:14,381] [ WARNING] - Compatibility Warning: The params of ChunkEvaluator.compute has been modified. The old version is `inputs`, `lengths`, `predictions`, `labels` while the current version is `lengths`, `predictions`, `labels`.  Please update the usage.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B-ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: I-ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/seqeval/metrics/sequence

eval precision: 0.952970 - recall: 0.945275 - f1: 0.949107
epoch:1 - step:400 - loss: 0.039633
epoch:1 - step:500 - loss: 0.024140
epoch:1 - step:600 - loss: 0.008022
epoch:1 - step:700 - loss: 0.013613
eval precision: 0.955716 - recall: 0.965909 - f1: 0.960785
epoch:2 - step:800 - loss: 0.038759
epoch:2 - step:900 - loss: 0.004242
epoch:2 - step:1000 - loss: 0.011100
eval precision: 0.954720 - recall: 0.968900 - f1: 0.961757
epoch:3 - step:1100 - loss: 0.017343
epoch:3 - step:1200 - loss: 0.012667
epoch:3 - step:1300 - loss: 0.016701
epoch:3 - step:1400 - loss: 0.006467
eval precision: 0.967151 - recall: 0.968501 - f1: 0.967825
epoch:4 - step:1500 - loss: 0.003037
epoch:4 - step:1600 - loss: 0.006062
epoch:4 - step:1700 - loss: 0.010095
eval precision: 0.966643 - recall: 0.967703 - f1: 0.967173
epoch:5 - step:1800 - loss: 0.009145
epoch:5 - step:1900 - loss: 0.007032
epoch:5 - step:2000 - loss: 0.007952
epoch:5 - step:2100 - loss: 0.009969
eval precision: 0.966780 - recall: 0.963118 -

## 模型预测

训练保存好的模型，即可用于预测。如以下示例代码自定义预测数据，调用`predict()`函数即可一键预测。

In [6]:
preds = my_predict(model, test_loader, test_ds, label_vocab)
file_path = "bert_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
# Print some examples
print(
    "The results have been saved in the file: %s, some examples are shown below: "
    % file_path)
print("\n".join(preds[:10]))

The results have been saved in the file: bert_results.txt, some examples are shown below: 
('中共中央', 'ORG')('致', 'O')('中国致公党十一大', 'ORG')('的', 'O')('贺', 'O')('词', 'O')('各', 'O')('位', 'O')('代', 'O')('表', 'O')('、', 'O')('各', 'O')('位', 'O')('同', 'O')('志', 'O')('：', 'O')('在', 'O')('中国致公党第十一次全国代表大会', 'ORG')('隆', 'O')('重', 'O')('召', 'O')('开', 'O')('之', 'O')('际', 'O')('，', 'O')('中国共产党中央委员会', 'ORG')('谨', 'O')('向', 'O')('大', 'O')('会', 'O')('表', 'O')('示', 'O')('热', 'O')('烈', 'O')('的', 'O')('祝', 'O')('贺', 'O')('，', 'O')('向', 'O')('致公党', 'ORG')('的', 'O')('同', 'O')('志', 'O')('们', 'O')('致', 'O')('以', 'O')('亲', 'O')('切', 'O')('的', 'O')('问', 'O')('候', 'O')('！', 'O')
('在', 'O')('过', 'O')('去', 'O')('的', 'O')('五', 'O')('年', 'O')('中', 'O')('，', 'O')('致公党', 'ORG')('在', 'O')('邓小平', 'PER')('理', 'O')('论', 'O')('指', 'O')('引', 'O')('下', 'O')('，', 'O')('遵', 'O')('循', 'O')('社', 'O')('会', 'O')('主', 'O')('义', 'O')('初', 'O')('级', 'O')('阶', 'O')('段', 'O')('的', 'O')('基', 'O')('本', 'O')('路', 'O')('线', 'O')('，', 'O')('努',

# 进一步使用CRF

PaddleNLP提供了CRF Layer，它能够学习label之间的关系，能够帮助模型更好地学习、预测序列标注任务。

我们在PaddleNLP仓库中提供了[示例](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/examples/information_extraction/waybill_ie/run_ernie_crf.py)，您可以参照示例代码使用Ernie-CRF结构完成快递单信息抽取任务。

# 加入交流群，一起学习吧

现在就加入课程QQ交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >